# **1. Load python packages and dataset**

In [1]:
# LIBRARIES -----
# !pip install xlsxwriter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Set seaborn style
sns.set(style="whitegrid")

from scipy.stats import boxcox
from sklearn.impute import SimpleImputer

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
#Mount google drive folder
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#read the transactions data
df = pd.read_csv('/content/drive/MyDrive/Lending_Club_Analysis/Output_files/dataframe_for_eda.csv', low_memory=False)

In [4]:
df.shape

(1345310, 93)

In [5]:
#First 5 rows of data
df.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,68407277,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,Mar-2019,564.0,560.0,0.0,1.0,Individual,0.0,722.0,144904.0,9300.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,4.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,N,Cash,N
1,68355089,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,Mar-2019,699.0,695.0,0.0,1.0,Individual,0.0,0.0,204396.0,111800.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,0.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,N,Cash,N
2,68341763,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,MORTGAGE,63000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,home_improvement,NaN,605xx,IL,10.78,0.0,Aug-2000,695.0,699.0,0.0,6.0,0.0,7869.0,56.2,18.0,w,0.0,0.0,22705.924294,22705.92,20000.0,2705.92,0.0,0.0,0.0,Jun-2017,15813.30,Mar-2019,704.0,700.0,0.0,1.0,Joint App,0.0,0.0,189699.0,14000.0,6.0,31617.0,2737.0,55.9,0.0,0.0,125.0,184.0,14.0,14.0,5.0,101.0,10.0,0.0,2.0,3.0,2.0,4.0,6.0,4.0,7.0,3.0,6.0,0.0,0.0,0.0,0.0,100.0,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,N,Cash,N
3,68476807,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,3 years,MORTGAGE,104433.0,Source Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,major_purchase,Major purchase,174xx,PA,25.37,1.0,Jun-1998,695.0,699.0,3.0,12.0,0.0,21929.0,64.5,35.0,w,0.0,0.0,11740.500000,11740.50,10400.0,1340.50,0.0,0.0,0.0,Jul-2016,10128.96,Mar-2018,704.0,700.0,0.0,1.0,Individual,0.0,0.0,331730.0,34000.0,10.0,27644.0,4567.0,77.5,0.0,0.0,128.0,210.0,4.0,4.0,6.0,4.0,1.0,0.0,4.0,6.0,5.0,9.0,10.0,7.0,19.0,6.0,12.0,0.0,0.0,0.0,4.0,96.6,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,N,Cash,N
4,68426831,11950.0,11950.0,11950.0,36 months,13.44,405.18,C,C3,Veterinary Tecnician,4 years,RENT,34000.0,Source Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,300

# **Data Preprocessing**

In [6]:
# Function to keep specified columns
def keep_relevant_columns(df, columns_to_keep):
    return df[columns_to_keep + ['loan_status']]

# Function to create categorical features based on given ranges
def create_categorical_feature(df, column, bins, labels):
    df[f'{column}_category'] = pd.cut(df[column], bins=bins, labels=labels, include_lowest=True)
    return df

# Function to impute missing values
def impute_missing_values(df, impute_dict):
    for column, strategy in impute_dict.items():
        imputer = SimpleImputer(strategy=strategy)
        df[column] = imputer.fit_transform(df[[column]])
    return df

# Function to create a binary feature based on a condition and impute NaN values
def create_binary_feature_and_impute(df, column, condition_value):
    df[f'{column}_binary'] = df[column].apply(lambda x: 'Y' if x >= condition_value else ('N' if x == 0 else x))
    most_frequent_status = df.loc[df['loan_status'] == 'Fully Paid', f'{column}_binary'].mode()[0]
    df[f'{column}_binary'].fillna(most_frequent_status, inplace=True)
    df.drop(column, axis=1, inplace=True)
    return df

# Function to apply Box-Cox transformation
def apply_box_cox_transformation(df, columns):
    for column in columns:
        if (df[column] <= 0).any():
            print(f"Column '{column}' contains non-positive values. Adding a small constant to make all values positive.")
            df[column] = df[column] + abs(df[column].min()) + 1  # Shift all values to be positive
        df[column], _ = boxcox(df[column])  # Apply Box-Cox transformation
    return df

# Function to process 46 variables without missing values
def process_no_missing_values(df):
    # Debug statement to check columns
    print("Columns before processing no missing values:", df.columns)

    # Convert delinq_2yrs to categorical if it exists
    if 'delinq_2yrs' in df.columns:
        df['delinq_2yrs_category'] = df['delinq_2yrs'].apply(lambda x: 'Yes' if x >= 1 else 'No')
        df.drop('delinq_2yrs', axis=1, inplace=True)
    else:
        print("Column 'delinq_2yrs' not found.")

    # Calculate percentage of loan paid if total_rec_prncp exists
    if 'total_rec_prncp' in df.columns and 'loan_amnt' in df.columns:
        df['perc_loan_paid'] = df['total_rec_prncp'] / df['loan_amnt'] * 100
        df.drop('total_rec_prncp', axis=1, inplace=True)
    else:
        print("Columns 'total_rec_prncp' or 'loan_amnt' not found.")

    # Convert recoveries to categorical if it exists
    if 'recoveries' in df.columns:
        df['recoveries_category'] = df['recoveries'].apply(lambda x: 'Y' if pd.notnull(x) and x > 0 else 'N')
        df.drop('recoveries', axis=1, inplace=True)
    else:
        print("Column 'recoveries' not found.")

    # List of columns to keep
    columns_to_keep = [
        'id', 'loan_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade',
        'home_ownership', 'annual_inc', 'loan_status', 'purpose', 'delinq_2yrs_category', 'open_acc', 'revol_bal',
        'total_acc', 'perc_loan_paid', 'recoveries_category', 'last_fico_range_high', 'last_fico_range_low', 'debt_settlement_flag'
    ]

    # Filter the DataFrame to keep only the specified columns
    df_filtered = keep_relevant_columns(df, columns_to_keep)

    # Debug statement to check columns
    print("Columns after filtering:", df_filtered.columns)

    # Create categorical features based on given ranges
    df_filtered = create_categorical_feature(df_filtered, 'int_rate', [-float('inf'), 5, 10, 15, 20, 25, float('inf')],
                                             ['<=5%', '>5% - <=10%', '>10% - <=15%', '>15% - <=20%', '>20% - <=25%', '>25%'])
    df_filtered = create_categorical_feature(df_filtered, 'installment', [-float('inf'), 5, 405, 707, 1719, float('inf')],
                                             ['No installment', 'Low', 'Medium', 'High', 'Very High'])
    df_filtered = create_categorical_feature(df_filtered, 'annual_inc', [-float('inf'), 0, 74000, 125000, 325000, float('inf')],
                                             ['No Annual Income', 'Low', 'Medium', 'High', 'Very High'])
    df_filtered = create_categorical_feature(df_filtered, 'revol_bal', [-float('inf'), 0, 17404, 32425, 134250, float('inf')],
                                             ['No revolving balance', 'Low', 'Medium', 'High', 'Very High'])
    df_filtered = create_categorical_feature(df_filtered, 'last_fico_range_high', [300, 579, 669, 739, 799, 850],
                                             ['Poor', 'Fair', 'Good', 'Very Good', 'Exceptional'])
    df_filtered = create_categorical_feature(df_filtered, 'last_fico_range_low', [300, 579, 669, 739, 799, 850],
                                             ['Poor', 'Fair', 'Good', 'Very Good', 'Exceptional'])

    # Replace home_ownership values
    df_filtered['home_ownership'] = df_filtered['home_ownership'].replace(['ANY', 'OTHER', 'NONE'], 'OTHERS')

    return df_filtered

# Main preprocessing function for both sets of variables
def preprocess_data(df):
    # List of columns to keep for the first set of variables
    columns_to_keep = [
        'dti', 'inq_last_6mths', 'revol_util', 'tot_cur_bal', 'avg_cur_bal', 'bc_open_to_buy',
        'mort_acc', 'num_actv_rev_tl', 'percent_bc_gt_75', 'tot_hi_cred_lim', 'total_bal_ex_mort',
        'total_bc_limit', 'total_il_high_credit_limit'
    ]

    # Keep relevant columns
    df_filtered = keep_relevant_columns(df, columns_to_keep)

    # Create categorical features for the first set of variables
    bins_dict = {
        'tot_cur_bal': [-1, 0, 80231, 340365, 806223, float('inf')],
        'avg_cur_bal': [-1, 0, 6057, 32220, 90154, float('inf')],
        'total_bal_ex_mort': [-1, 0, 45393, 99568, 287817, float('inf')],
        'total_bc_limit': [-1, 0, 19100, 47200, 123000, float('inf')],
        'total_il_high_credit_limit': [-1, 0, 44594, 90692, 245878, float('inf')]
    }

    labels = ['No Balance', 'Low', 'Medium', 'High', 'Very High']

    for column, bins in bins_dict.items():
        df_filtered = create_categorical_feature(df_filtered, column, bins, labels)

    # Create binary feature and impute NaN values
    df_filtered = create_binary_feature_and_impute(df_filtered, 'inq_last_6mths', 1)

    # Impute missing values for the first set of variables
    impute_dict = {
        'dti': 'median',
        'revol_util': 'median',
        'tot_cur_bal': 'median',
        'avg_cur_bal': 'median',
        'bc_open_to_buy': 'median',
        'mort_acc': 'most_frequent',
        'num_actv_rev_tl': 'most_frequent',
        'percent_bc_gt_75': 'most_frequent',
        'tot_hi_cred_lim': 'median',
        'total_bal_ex_mort': 'median',
        'total_bc_limit': 'median',
        'total_il_high_credit_limit': 'median'
    }

    df_filtered = impute_missing_values(df_filtered, impute_dict)

    # Apply Box-Cox transformation for the first set of variables
    columns_to_transform = [
        'dti', 'tot_cur_bal', 'avg_cur_bal', 'bc_open_to_buy', 'mort_acc',
        'num_actv_rev_tl', 'percent_bc_gt_75', 'tot_hi_cred_lim', 'total_bal_ex_mort',
        'total_bc_limit', 'total_il_high_credit_limit'
    ]

    df_filtered = apply_box_cox_transformation(df_filtered, columns_to_transform)

    # Process the second set of variables (46 variables with no missing values)
    df_no_missing_values_processed = process_no_missing_values(df)

    # Combine both processed DataFrames
    df_combined = pd.concat([df_filtered, df_no_missing_values_processed], axis=1)

    # Remove the first occurrence of 'loan_status'
    positions = [i for i, col in enumerate(df_combined.columns) if col == 'loan_status']
    if len(positions) > 1:
        df_combined = df_combined.loc[:, ~((df_combined.columns == 'loan_status') & (df_combined.columns.duplicated()))]

    # Move 'id' to be the first column and 'loan_status' to be the last column
    columns = df_combined.columns.tolist()
    columns.insert(0, columns.pop(columns.index('id')))
    columns.append(columns.pop(columns.index('loan_status')))
    df_combined = df_combined[columns]

    return df_combined

In [7]:
# Preprocess data
df_preprocessed = preprocess_data(df)

# Display the first few rows of the preprocessed DataFrame
df_preprocessed.head()

<ipython-input-6-5c01b27bd216>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{column}_category'] = pd.cut(df[column], bins=bins, labels=labels, include_lowest=True)
<ipython-input-6-5c01b27bd216>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{column}_category'] = pd.cut(df[column], bins=bins, labels=labels, include_lowest=True)
<ipython-input-6-5c01b27bd216>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

Column 'dti' contains non-positive values. Adding a small constant to make all values positive.


<ipython-input-6-5c01b27bd216>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column], _ = boxcox(df[column])  # Apply Box-Cox transformation


Column 'tot_cur_bal' contains non-positive values. Adding a small constant to make all values positive.
Column 'avg_cur_bal' contains non-positive values. Adding a small constant to make all values positive.
Column 'bc_open_to_buy' contains non-positive values. Adding a small constant to make all values positive.
Column 'mort_acc' contains non-positive values. Adding a small constant to make all values positive.
Column 'num_actv_rev_tl' contains non-positive values. Adding a small constant to make all values positive.
Column 'percent_bc_gt_75' contains non-positive values. Adding a small constant to make all values positive.
Column 'tot_hi_cred_lim' contains non-positive values. Adding a small constant to make all values positive.
Column 'total_bal_ex_mort' contains non-positive values. Adding a small constant to make all values positive.
Column 'total_bc_limit' contains non-positive values. Adding a small constant to make all values positive.
Column 'total_il_high_credit_limit' contai

<ipython-input-6-5c01b27bd216>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{column}_category'] = pd.cut(df[column], bins=bins, labels=labels, include_lowest=True)
<ipython-input-6-5c01b27bd216>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{column}_category'] = pd.cut(df[column], bins=bins, labels=labels, include_lowest=True)
<ipython-input-6-5c01b27bd216>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

,id,dti,revol_util,tot_cur_bal,avg_cur_bal,bc_open_to_buy,mort_acc,num_actv_rev_tl,percent_bc_gt_75,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,tot_cur_bal_category,avg_cur_bal_category,total_bal_ex_mort_category,total_bc_limit_category,total_il_high_credit_limit_category,inq_last_6mths_binary,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,home_ownership,annual_inc,purpose,delinq_2yrs_category,open_acc,revol_bal,total_acc,perc_loan_paid,recoveries_category,last_fico_range_high,last_fico_range_low,debt_settlement_flag,int_rate_category,installment_category,annual_inc_category,revol_bal_category,last_fico_range_high_category,last_fico_range_low_category,loan_status
0,68407277,3.221603,29.7,36.003101,16.738945,16.837260,0.604321,1.743244,0.000000,26.552149,27.971575,30.868135,82.238313,Medium,Medium,Low,Low,Low,Y,3600.0,3600.0,36 months,13.99,123.03,C,MORTGAGE,55000.0,debt_consolidation,No,7.0,2765.0,13.0,100.0,N,564.0,560.0,N,>10% - <=15%,Low,Low,Low,Poor,Poor,Fully Paid
1,68355089,5.464799,19.2,38.413155,14.827919,40.713790,1.182337,1.958617,3.417456,28.955689,41.947603,93.869043,102.112178,Medium,Medium,Low,High,Low,Y,24700.0,24700.0,36 months,11.99,820.28,C,MORTGAGE,65000.0,small_business,Yes,22.0,21470.0,38.0,100.0,N,699.0,695.0,N,>10% - <=15%,High,Low,Medium,Good,Good,Fully Paid
2,68341763,4.434156,56.2,37.878917,17.874444,19.642289,1.273895,1.484831,9.428180,27.399525,34.911829,42.074228,84.711671,Medium,Medium,Low,Low,Low,N,20000.0,20000.0,60 months,10.78,432.66,B,MORTGAGE,63000.0,home_improvement,No,6.0,7869.0,18.0,100.0,N,704.0,700.0,N,>10% - <=15%,Medium,Low,Low,Good,Good,Fully Paid
3,68476807,6.908813,64.5,42.041101,17.509330,22.332893,1.346200,2.143737,10.304290,30.457479,51.934193,61.374403,162.821420,Medium,Medium,Medium,Medium,Medium,Y,10400.0,10400.0,60 months,22.45,289.91,F,MORTGAGE,104433.0,major_purchase,Yes,12.0,21929.0,35.0,100.0,N,704.0,700.0,N,>20% - <=25%,Low,Medium,Medium,Good,Good,Fully Paid
4,68426831,4.306306,68.4,22.323477,11.771486,14.425313,0.000000,1.484831,13.132012,18.115409,31.761285,48.083118,51.838673,Low,Low,Low,Low,Low,N,11950.0,11950.0,36 months,13.44,405.18,C,RENT,34000.0,debt_consolidation,No,5.0,8822.0,6.0,100.0,N,759.0,755.0,N,>10% - <=15%,Medium,Low,Low,Very Good,Very Good,Fully Paid


In [8]:
df_preprocessed.shape

(1345310, 44)

In [9]:
df_preprocessed.head()

,id,dti,revol_util,tot_cur_bal,avg_cur_bal,bc_open_to_buy,mort_acc,num_actv_rev_tl,percent_bc_gt_75,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,tot_cur_bal_category,avg_cur_bal_category,total_bal_ex_mort_category,total_bc_limit_category,total_il_high_credit_limit_category,inq_last_6mths_binary,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,home_ownership,annual_inc,purpose,delinq_2yrs_category,open_acc,revol_bal,total_acc,perc_loan_paid,recoveries_category,last_fico_range_high,last_fico_range_low,debt_settlement_flag,int_rate_category,installment_category,annual_inc_category,revol_bal_category,last_fico_range_high_category,last_fico_range_low_category,loan_status
0,68407277,3.221603,29.7,36.003101,16.738945,16.837260,0.604321,1.743244,0.000000,26.552149,27.971575,30.868135,82.238313,Medium,Medium,Low,Low,Low,Y,3600.0,3600.0,36 months,13.99,123.03,C,MORTGAGE,55000.0,debt_consolidation,No,7.0,2765.0,13.0,100.0,N,564.0,560.0,N,>10% - <=15%,Low,Low,Low,Poor,Poor,Fully Paid
1,68355089,5.464799,19.2,38.413155,14.827919,40.713790,1.182337,1.958617,3.417456,28.955689,41.947603,93.869043,102.112178,Medium,Medium,Low,High,Low,Y,24700.0,24700.0,36 months,11.99,820.28,C,MORTGAGE,65000.0,small_business,Yes,22.0,21470.0,38.0,100.0,N,699.0,695.0,N,>10% - <=15%,High,Low,Medium,Good,Good,Fully Paid
2,68341763,4.434156,56.2,37.878917,17.874444,19.642289,1.273895,1.484831,9.428180,27.399525,34.911829,42.074228,84.711671,Medium,Medium,Low,Low,Low,N,20000.0,20000.0,60 months,10.78,432.66,B,MORTGAGE,63000.0,home_improvement,No,6.0,7869.0,18.0,100.0,N,704.0,700.0,N,>10% - <=15%,Medium,Low,Low,Good,Good,Fully Paid
3,68476807,6.908813,64.5,42.041101,17.509330,22.332893,1.346200,2.143737,10.304290,30.457479,51.934193,61.374403,162.821420,Medium,Medium,Medium,Medium,Medium,Y,10400.0,10400.0,60 months,22.45,289.91,F,MORTGAGE,104433.0,major_purchase,Yes,12.0,21929.0,35.0,100.0,N,704.0,700.0,N,>20% - <=25%,Low,Medium,Medium,Good,Good,Fully Paid
4,68426831,4.306306,68.4,22.323477,11.771486,14.425313,0.000000,1.484831,13.132012,18.115409,31.761285,48.083118,51.838673,Low,Low,Low,Low,Low,N,11950.0,11950.0,36 months,13.44,405.18,C,RENT,34000.0,debt_consolidation,No,5.0,8822.0,6.0,100.0,N,759.0,755.0,N,>10% - <=15%,Medium,Low,Low,Very Good,Very Good,Fully Paid


In [10]:
# Export the preprocessed DataFrame to Google Drive
output_path = '/content/drive/MyDrive/Lending_Club_Analysis/Output_files/preprocessed_data.csv'
df_preprocessed.to_csv(output_path, index=False)

print(f"DataFrame exported successfully to {output_path}")

DataFrame exported successfully to /content/drive/MyDrive/Lending_Club_Analysis/Output_files/preprocessed_data.csv
